In [ ]:
!pip install numpy
!pip install pandas
!pip install openpyxl
!pip install nltk
!pip install sklearn
!pip install fasttext

You should consider upgrading via the '/home/danial/PycharmProjects/MIR-HWs3-5/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/danial/PycharmProjects/MIR-HWs3-5/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
import os

import nltk
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


False

In [2]:
dfs = []
for file in os.listdir("./Excels"):
    df = pd.read_excel(os.path.join("./Excels", file))
    dfs.append(df)
    print(file, end=": ")
    print(df.shape)
df = pd.concat(dfs)

WWDC.xlsx: (2080, 19)
spaceX.xlsx: (2010, 19)
Google.xlsx: (2051, 19)
Meta.xlsx: (1644, 19)
metaverse.xlsx: (1553, 19)
Apple.xlsx: (2031, 19)
instagram.xlsx: (1949, 19)
Microsoft.xlsx: (2080, 19)
Twitter.xlsx: (1616, 19)
facebook.xlsx: (2027, 19)
Amazon.xlsx: (2078, 19)
nvidia.xlsx: (2010, 19)
Tesla.xlsx: (2004, 19)


In [3]:
df.columns, df.shape

(Index(['Tweet Id', 'Text', 'Name', 'Screen Name', 'UTC', 'Created At',
        'Favorites', 'Retweets', 'Language', 'Client', 'Tweet Type', 'URLs',
        'Hashtags', 'Mentions', 'Media Type', 'Media URLs', 'Unnamed: 16',
        'Unnamed: 17', 'Unnamed: 18'],
       dtype='object'),
 (25133, 19))

In [4]:
df = df[df['Language'] == 'en']
df.reset_index(drop=True, inplace=True)

In [5]:
from nltk.corpus import stopwords
import re
def text_to_words(text):
    text = text.lower()
    text = re.sub(
    "(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})",
    "", text)
    text = re.sub("#\w+", "", text)
    text = re.sub("@\w+", "", text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english') and word.isalpha()]
    return words
df['Text_words'] = df['Text'].apply(text_to_words)

In [6]:
nltk.FreqDist(df['Text_words'].explode()).most_common(20)

[('rt', 8640),
 ('amp', 1744),
 ('new', 1247),
 ('join', 1132),
 ('video', 1129),
 ('campaign', 924),
 ('microsoft', 837),
 ('us', 834),
 ('latest', 778),
 ('follow', 748),
 ('technology', 747),
 ('environments', 746),
 ('testing', 675),
 ('skills', 668),
 ('apple', 667),
 ('unique', 657),
 ('airdrop', 657),
 ('rtx', 655),
 ('mr', 649),
 ('heroes', 646)]

In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

vocabulary = list(set(df['Text_words'].explode()) - {np.nan})

tfidf = TfidfVectorizer(vocabulary=vocabulary)
tfidf_tran=tfidf.fit_transform(df['Text_words'].apply(lambda x: ' '.join(x)))


def gen_vector_T(tokens):
    Q = np.zeros((len(vocabulary)))
    x= tfidf.transform(tokens)
    for token in tokens:
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except Exception as e:
            print(e)
            pass
    return Q

def cosine_sim(a, b):
    if sum(a) == 0 or sum(b) == 0:
        return -1
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim


def cosine_similarity_T(k, query):
    tokens = text_to_words(query)
    print(tokens)
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0, 'q_clean'] = tokens

    d_cosines = []

    query_vector = gen_vector_T(tokens)
    for d in tfidf_tran.A:
        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]
    a = pd.DataFrame()
    for i, index in enumerate(out):
        a.loc[i, 'index'] = str(index)
        a.loc[i, 'text'] = df.iloc[index]['Text']
        a.loc[i, 'words'] = str(df.iloc[index]['Text_words'])
        a.loc[i, 'Score'] = d_cosines[int(index)]
    return a

/home/danial/PycharmProjects/MIR-HWs3-5/venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [12]:
cosine_similarity_T(10, "elon musk")

['elon', 'musk']


,index,text,words,Score
0,1817,@SpaceX + @DogelonMars = 🚀\n\n@elonmusk Don't...,['elon'],1.000000
1,1811,RT @ElonMartians : @SpaceX + @DogelonMars = 🚀\...,"['rt', 'elon']",0.949679
2,1813,RT @ElonMartians : @SpaceX + @DogelonMars = 🚀\...,"['rt', 'elon']",0.949679
3,1814,RT @ElonMartians : @SpaceX + @DogelonMars = 🚀\...,"['rt', 'elon']",0.949679
4,15901,@elonmusk @elonamuskjet @ElonMuskNewsOrg @Tesl...,"['elon', 'spacex']",0.668938
5,2813,"RT @Dyoungelonhero : Elon,Send #GalaxyHeroes t...","['rt', 'elon', 'send']",0.615394
6,2583,"RT @Dyoungelonhero : Elon,Send #GalaxyHeroes t...","['rt', 'elon', 'send']",0.615394
7,2158,"RT @Dyoungelonhero : Elon,Send #GalaxyHeroes t...","['rt', 'elon', 'send']",0.615394
8,2131,In Elon We Trust! ❤️🙏🏼 #dogecoin #tesla #Spac...,"['elon', 'trust']",0.530308
9,14730,Could Elon Musk be a thief? Elon Musk why don'...,"['could', 'elon', 'musk', 'thief', 'elon', 'mu...",0.485152


In [39]:
vocabulary = list(set(df['Text_words'].explode()) - {np.nan})
vocabulary_index = {word: idx for idx, word in enumerate(vocabulary)}

mat = np.zeros((len(df), len(vocabulary)), )

for index, row in df.iterrows():
    for token in row['Text_words']:
        mat[index][vocabulary_index[token]] = 1


def boolean_search(k, query):
    #print("Cosine Similarity")
    tokens = text_to_words(query)
    query_vector = []
    for token in tokens:
        if token in vocabulary_index:
            query_vector.append(vocabulary_index[token])
    out = []
    for index, vec in enumerate(mat):
        flag = True
        for token_index in query_vector:
            if vec[token_index] == 0:
                flag = False
                break
        if flag:
            out.append(index)
        if len(out) == k:
            break
    print(out)
    a = pd.DataFrame()
    for i, index in enumerate(out):
        a.loc[i, 'index'] = str(index)
        a.loc[i, 'text'] = df.iloc[index]['Text']
        a.loc[i, 'words'] = str(df.iloc[index]['Text_words'])

    return a

In [40]:
boolean_search(10, "Elon Musk")

[22, 93, 350, 1495, 1517, 1531, 1543, 1547, 1548, 1549]


,index,text,words
0,22,Elon Musk give me MONEY bitch And don't forge...,"['elon', 'musk', 'give', 'money', 'bitch', 'fo..."
1,93,"Elon Musk give me $10,000 AND Putin a giant s...","['elon', 'musk', 'give', 'putin', 'giant', 'sp..."
2,350,"Elon Musk give me $99,000 AND Putin elegant a...","['elon', 'musk', 'give', 'putin', 'elegant', '..."
3,1495,what if Gerald Cotten is Elon Musk (:)\n@elonm...,"['gerald', 'cotten', 'elon', 'musk', 'musk']"
4,1517,Elon Musk's SpaceX has cleared a huge obstacle...,"['elon', 'musk', 'spacex', 'cleared', 'huge', ..."
5,1531,How China could kill Space X.\nSpace X's space...,"['china', 'could', 'kill', 'space', 'space', '..."
6,1543,RT @weatherindia : Elon Musk's #SpaceX has rec...,"['rt', 'elon', 'musk', 'received', 'environmen..."
7,1547,RT @TweakTown : Elon Musk says @SpaceX's Stars...,"['rt', 'elon', 'musk', 'says', 'starship', 'ro..."
8,1548,RT @TweakTown : Elon Musk says @SpaceX's Stars...,"['rt', 'elon', 'musk', 'says', 'starship', 'ro..."
9,1549,RT @TweakTown : Elon Musk says @SpaceX's Stars...,"['rt', 'elon', 'musk', 'says', 'starship', 'ro..."
